In [1]:
# pyarrow ou fastparquet
import pandas as pd
import numpy as np
from typing import Literal

In [2]:
full_history = pd.read_parquet('./data/full_history.parquet')

rastro_contratos = pd.read_parquet('./data/rastro_contratos.parquet')

In [3]:
display(full_history.head(3))
display(rastro_contratos.head(10))

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15


,id_antigo,id_novo,data_evento
0,10000003,10002500,2015-02-01
1,10000047,10002501,2015-02-01
2,10000048,10002502,2015-02-01
3,10000064,10002503,2015-02-01
4,10000081,10002504,2015-02-01
5,10000110,10002505,2015-02-01
6,10000164,10002506,2015-02-01
7,10000169,10002507,2015-02-01
8,10000170,10002508,2015-02-01
9,10000185,10002509,2015-02-01


### Verificação 1: IDs devem ser strings ou inteiros coerentes

In [73]:
assert full_history['id_contrato'].notna().all(), "❌ Existem contratos nulos em full_history"
assert rastro_contratos['id_antigo'].notna().all(), "❌ Existem IDs antigos nulos"
assert rastro_contratos['id_novo'].notna().all(), "❌ Existem IDs novos nulos"

#### Verifica se há conflitos de tipo (mistura de int/str)

In [74]:
# mapear tipos em full_history
tipos_ids_hist = full_history['id_contrato'].map(type).value_counts()

# mapear tipos em rastro_contratos
tipos_ids_rastro = pd.concat([
    rastro_contratos['id_antigo'].map(type),
    rastro_contratos['id_novo'].map(type)
]).value_counts()

print("Tipos encontrados em id_contrato:", tipos_ids_hist.to_dict())
print("Tipos encontrados em rastro_contratos:", tipos_ids_rastro.to_dict())

Tipos encontrados em id_contrato: {<class 'str'>: 115186}
Tipos encontrados em rastro_contratos: {<class 'str'>: 8246}


### Verificação 2: Datas devem estar em datetime64[ns]

In [75]:
assert pd.api.types.is_datetime64_any_dtype(full_history['data_ref']), "❌ 'data_ref' não está no formato datetime"

assert pd.api.types.is_datetime64_any_dtype(rastro_contratos['data_evento']), "❌ 'data_evento' não está no formato datetime"

### Verificação 3: Datas não devem ter valores faltantes

In [141]:
full_history.head(10)

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15
3,10000000,2014-09-30,2015-04-01,201501,30
4,10000000,2014-09-30,2015-05-01,201501,60
5,10000000,2014-09-30,2015-06-01,201501,90
6,10000000,2014-09-30,2015-07-01,201501,60
7,10000000,2014-09-30,2015-08-01,201501,90
8,10000000,2014-09-30,2015-09-01,201501,90
9,10000000,2014-09-30,2015-10-01,201501,60


In [76]:
assert full_history['data_ref'].notna().all(), "❌ Existem datas nulas em full_history['data_ref']"

assert rastro_contratos['data_evento'].notna().all(), "❌ Existem datas nulas em rastro_contratos['data_evento']"

### Verificação 4: Datas devem estar normalizadas (hora = 00:00:00)

In [77]:
# Para comparar datas corretamente, é fundamental que a parte de hora/minuto
# esteja "zerada", ou seja: datetime.time(0, 0). Isso evita erros sutis quando
# usamos comparações como: data_ref == data_evento

amostras_ref = full_history['data_ref'].dropna().dt.time.unique()
amostras_evt = rastro_contratos['data_evento'].dropna().dt.time.unique()

print("⏰ Horários únicos em data_ref:", amostras_ref)
print("⏰ Horários únicos em data_evento:", amostras_evt)

# Verificação de integridade das horas
hora_normal = pd.Timestamp("00:00:00").time()

ref_ok = all(t == hora_normal for t in amostras_ref)
evt_ok = all(t == hora_normal for t in amostras_evt)

if not ref_ok:
    print("⚠️ AVISO: 'data_ref' tem registros com horário diferente de 00:00:00")
if not evt_ok:
    print("⚠️ AVISO: 'data_evento' tem registros com horário diferente de 00:00:00")
if ref_ok and evt_ok:
    print("✅ Todas as datas estão normalizadas com horário 00:00:00 (seguro para comparação)")


⏰ Horários únicos em data_ref: [datetime.time(0, 0)]
⏰ Horários únicos em data_evento: [datetime.time(0, 0)]
✅ Todas as datas estão normalizadas com horário 00:00:00 (seguro para comparação)


# Testes de Continuidade

##### 1) se há algum contrato que salta no tempo

##### 2) construção da flag_acordo

##### 3) se algum contrato sofreu reaging

In [78]:
full_history.head()

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
0,10000000,2014-09-30,2015-01-01,201501,30
1,10000000,2014-09-30,2015-02-01,201501,15
2,10000000,2014-09-30,2015-03-01,201501,15
3,10000000,2014-09-30,2015-04-01,201501,30
4,10000000,2014-09-30,2015-05-01,201501,60


In [ ]:
# flowchart mermaid (markdown)

In [ ]:
from collections import Counter
from typing import Literal

def diagnostico_descontinuidade(
    df: pd.DataFrame,
    *,
    col_id: str = "id_contrato",
    col_data: str = "data_ref",
    freq: Literal["M", "Q", "A", "D", "H"] = "M",
    verbose: bool = True
) -> dict:
    """
    Verifica descontinuidades na sequência de `col_data` para cada `col_id`.

    Retorna
    -------
    dict com:
      contratos_com_gap : set   IDs com ao menos um salto
      n_contratos_gap   : int   quantidade desses IDs
      proporcao_gap     : float porcentagem sobre o total
      distrib_gaps      : Counter {tamanho_gap: ocorrências}
      df_gaps           : DataFrame com detalhes (id, n_gaps, tam_max_gap)
    """
    # 1) preparo --------------------------------------------------------
    df_tmp = df[[col_id, col_data]].dropna().copy()
    df_tmp[col_data] = pd.to_datetime(df_tmp[col_data]).dt.normalize()
    df_tmp.sort_values([col_id, col_data], inplace=True)

    registros, distrib = [], Counter()

    # 2) loop por contrato ---------------------------------------------
    for cid, g in df_tmp.groupby(col_id):
        datas = g[col_data]

        if freq in {"M", "Q", "A"}:
            # Converte para Period -> inteiro -> diff
            num = datas.dt.to_period(freq).astype(int)
            saltos = num.diff().fillna(0).astype(int)
        elif freq == "D":
            saltos = (datas.diff().dt.days.fillna(0).astype(int))
        elif freq == "H":
            saltos = (datas.diff().dt.total_seconds()
                               .fillna(0)
                               .div(3600)
                               .astype(int))
        else:
            raise ValueError(f"freq '{freq}' não suportada")

        gaps = saltos[saltos > 1]

        if not gaps.empty:
            registros.append({
                "id_contrato": cid,
                "n_gaps": len(gaps),
                "tam_max_gap": gaps.max()
            })
            distrib.update(gaps.values)

    # 3) sumariza -------------------------------------------------------

    # 3) sumariza -----------------------------------------------------------
    if registros:                               # ↔ há pelo menos 1 gap
        df_gaps = pd.DataFrame(registros)
        contratos_com_gap = set(df_gaps[col_id])
    else:                                       # ↔ nenhum gap encontrado
        df_gaps = pd.DataFrame(columns=[col_id, "n_gaps", "tam_max_gap"])
        contratos_com_gap = set()

    n_contratos_gap = len(contratos_com_gap)
    total_contratos = df_tmp[col_id].nunique()
    proporcao_gap   = n_contratos_gap / total_contratos if total_contratos else 0

    n_contratos_gap   = len(contratos_com_gap)
    total_contratos   = df_tmp[col_id].nunique()
    proporcao_gap     = n_contratos_gap / total_contratos if total_contratos else 0

    resumo = {
        "contratos_com_gap": contratos_com_gap,
        "n_contratos_gap": n_contratos_gap,
        "proporcao_gap": proporcao_gap,
        "distrib_gaps": distrib,
        "df_gaps": df_gaps
    }

    # 4) prints de auditoria -------------------------------------------
    if verbose:
        print("📋 Diagnóstico de continuidade")
        print(f"• Contratos analisados ............: {total_contratos:,}")
        print(f"• Contratos com gap ...............: {n_contratos_gap:,} "
              f"({proporcao_gap:.1%})")
        if distrib:
            maior = max(distrib)
            print(f"• Maior gap encontrado ............: {maior} período(s)")
            print("• Top tamanhos de gap (freq):")
            for tam, freq_ in sorted(distrib.items(), key=lambda x: -x[1])[:10]:
                print(f"    – {tam} → {freq_:,}")
            print("• Exemplos de contratos problemáticos:")
            print(df_gaps.sort_values("tam_max_gap", ascending=False)
                        .head(5)
                        .to_string(index=False))
        else:
            print("✅ Nenhum salto detectado – dados contínuos")

    return resumo


In [ ]:
res = diagnostico_descontinuidade(
    full_history,
    col_id="id_contrato",
    col_data="data_ref",
    freq="M")      # ou "D", "H"…

📋 Diagnóstico de continuidade
• Contratos analisados ............: 12,228
• Contratos com gap ...............: 0 (0.0%)
✅ Nenhum salto detectado – dados contínuos


In [81]:
full_history.columns.tolist()

['id_contrato', 'data_inicio_contrato', 'data_ref', 'safra', 'dias_atraso']

#### Criar gaps no histório para garantir confiabilidade na função anterior 

In [82]:
import numpy as np
import pandas as pd
from numpy.random import default_rng

def inserir_gaps_aleatorios(
    df: pd.DataFrame,
    *,
    col_id: str = "id_contrato",
    col_data: str = "data_ref",
    frac_contratos: float = 0.20,      # % de contratos que receberão gap
    mean_gap: float = 2.0,             # média desejada (meses)
    max_gap: int = 5,                  # limite superior
    random_state: int | None = 42
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Remove linhas para criar gaps aleatórios em `data_ref`.

    Retorna
    -------
    df_gap      : DataFrame com registros faltantes simulados
    df_inj_info : DataFrame detalhando contrato, início e tamanho do gap
    """
    rng = default_rng(random_state)
    df_gap = df.copy()

    # 1. Seleciona aleatoriamente um subconjunto de contratos
    contratos = df[col_id].unique()
    n_select  = int(len(contratos) * frac_contratos)
    contratos_sel = rng.choice(contratos, size=n_select, replace=False)

    injec_log = []  # para registrar o que foi removido

    # 2. Processa cada contrato escolhido
    for cid in contratos_sel:
        g = df_gap.loc[df_gap[col_id] == cid].copy()
        datas = g[col_data].sort_values().dt.normalize().unique()

        if len(datas) < 8:          # pouco histórico? pula
            continue

        # 2a. Sorteia tamanho do gap (N~Normal, truncado entre 1 e max_gap)
        gap_len = int(
            np.clip(
                rng.normal(loc=mean_gap, scale=1.0),
                1, max_gap
            ).round()
        )

        # 2b. Escolhe ponto de início possível
        # (garante que existam gap_len meses consecutivos)
        idx_max_start = len(datas) - gap_len
        if idx_max_start <= 0:
            continue
        start_idx = rng.integers(0, idx_max_start)
        gap_start = datas[start_idx]

        # 2c. Constrói range mensal e remove
        gap_period = pd.date_range(gap_start, periods=gap_len, freq="MS")
        mask_remover = (df_gap[col_id] == cid) & (df_gap[col_data].isin(gap_period))
        df_gap = df_gap.loc[~mask_remover]

        injec_log.append({
            "id_contrato": cid,
            "gap_len": gap_len,
            "gap_start": gap_start
        })

    df_inj_info = pd.DataFrame(injec_log)
    return df_gap, df_inj_info


In [83]:
df_com_gaps, info = inserir_gaps_aleatorios(
    full_history,
    col_id="id_contrato",
    col_data="data_ref",
    frac_contratos=0.15,   # 15 % dos contratos afetados
    random_state=0
)

display(info.head())         # mostra onde e quanto foi removido

,id_contrato,gap_len,gap_start
0,10005898,1,2017-10-01
1,10005519,1,2016-09-01
2,10004460,3,2015-11-01
3,10005930,1,2016-06-01
4,10007484,3,2017-08-01


In [84]:
res = diagnostico_descontinuidade(
    df_com_gaps,
    col_id="id_contrato",
    col_data="data_ref",
    freq="M")      # ou "D", "H"…

📋 Diagnóstico de continuidade
• Contratos analisados ............: 12,228
• Contratos com gap ...............: 838 (6.9%)
• Maior gap encontrado ............: 6 período(s)
• Top tamanhos de gap (freq):
    – 3 → 301
    – 2 → 279
    – 4 → 198
    – 5 → 58
    – 6 → 2
• Exemplos de contratos problemáticos:
id_contrato  n_gaps  tam_max_gap
   10003600       1            6
   10006391       1            6
   10007533       1            5
   10007604       1            5
   10006213       1            5


In [85]:
df_com_gaps[df_com_gaps['id_contrato']=='10000140']

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
1319,10000140,2014-07-23,2015-01-01,201501,0
1320,10000140,2014-07-23,2015-02-01,201501,0
1323,10000140,2014-07-23,2015-05-01,201501,15
1324,10000140,2014-07-23,2015-06-01,201501,30
1325,10000140,2014-07-23,2015-07-01,201501,30
1326,10000140,2014-07-23,2015-08-01,201501,30
1327,10000140,2014-07-23,2015-09-01,201501,30
1328,10000140,2014-07-23,2015-10-01,201501,30
1329,10000140,2014-07-23,2015-11-01,201501,60


# Construção de flag_acordo

```mermaid
flowchart TD
    A[Inicio] --> B[Copiar dados e normalizar IDs e datas]
    B --> C[Construir dicionario de origem dos contratos]
    C --> D[Mapear id_original no historico]
    D --> E[Preparar eventos de renegociacao]
    E --> F[Mesclar eventos ao historico]
    F --> G[Criar flag_acordo para data exata do evento]
    G --> H[Reorganizar colunas para melhor leitura]
    H --> Z[Fim com DataFrame atualizado]

```

In [ ]:
# cuidado com cópias de dataframes em ambiente de produção (custo de alocação de memória adicional)
# em ambiente de experimentação não há problema 
# cópia é bacana quando você quer evitar efeitos colaterais inesperados, ou quando segurança e legibilidade pesam mais do que performance

def criar_flag_acordo(
    full_history: pd.DataFrame,
    rastro_contratos: pd.DataFrame,
    *,
    col_id_hist: str = "id_contrato",       # coluna-ID no histórico
    col_id_antigo: str = "id_antigo",       # ID antigo no rastro
    col_id_novo: str = "id_novo",           # ID novo   no rastro
    col_evento: str = "data_evento",        # data em que o novo contrato nasce
    col_data_ref: str = "data_ref",         # data da linha no histórico
    verbose: bool = True
) -> pd.DataFrame:
    """
    Marca flag_acordo = 1 NO CONTRATO ANTIGO, no mês imediatamente
    anterior ao nascimento do contrato novo (data_evento – 1 mês).
    Também cria id_original (primeiro contrato da cadeia).

    Retorna uma cópia de full_history com:
        • id_original
        • data_evento  (replicada do rastro)
        • flag_acordo  (0/1)
    """
    # ── 1. Normalização ───────────────────────────────────────────────
    fh = full_history.copy()
    rc = rastro_contratos.copy()

    fh[col_id_hist]   = fh[col_id_hist].astype(str).str.strip()
    rc[col_id_antigo] = rc[col_id_antigo].astype(str).str.strip()
    rc[col_id_novo]   = rc[col_id_novo].astype(str).str.strip()

    fh[col_data_ref] = pd.to_datetime(fh[col_data_ref], errors="coerce")\
                           .dt.normalize()
    rc[col_evento]   = pd.to_datetime(rc[col_evento], errors="coerce")\
                           .dt.normalize()

    # ── 2. id_original (segue cadeia id_novo → id_antigo) ─────────────
    link = dict(zip(rc[col_id_novo], rc[col_id_antigo]))

    def _orig(cid: str) -> str:
        while cid in link:
            cid = link[cid]
        return cid

    fh["id_original"] = fh[col_id_hist].map(_orig)

    # ── 3. Flag de acordo no contrato antigo ──────────────────────────
    eventos = rc[[col_id_antigo, col_evento]].rename(
        columns={col_id_antigo: col_id_hist}
    )
    # mês em que a flag deve ser marcada = evento – 1 mês
    eventos["data_flag"] = eventos[col_evento] - pd.offsets.MonthBegin(1)

    # junta data_evento e data_flag ao histórico (apenas pelo contrato antigo)
    fh = fh.merge(eventos[[col_id_hist, col_evento, "data_flag"]],
                  how="left", on=col_id_hist)

    fh["flag_acordo"] = (fh[col_data_ref] == fh["data_flag"]).astype(int)
    fh.drop(columns="data_flag", inplace=True)      # coluna auxiliar

    # ── 4. Organização de colunas ─────────────────────────────────────
    base_cols = [
        "id_original",
        col_id_hist,
        "data_inicio_contrato",
        col_data_ref,
        "safra",
        "dias_atraso",
        col_evento,        # agora presente para auditoria
        "flag_acordo",
    ]
    outras = [c for c in fh.columns if c not in base_cols]
    fh = fh[base_cols + outras]

    # ── 5. Auditoria opcional ─────────────────────────────────────────
    if verbose:
        total_flags = int(fh["flag_acordo"].sum())
        contratos_flag = fh.loc[fh["flag_acordo"] == 1, col_id_hist].nunique()
        print("📌 criar_flag_acordo — auditoria")
        print(f"• Flags marcadas ...............: {total_flags:,}")
        print(f"• Contratos com flag_acordo=1 ..: {contratos_flag:,}")
        display(
            fh.loc[fh["flag_acordo"] == 1,
                   [col_id_hist, col_data_ref, "dias_atraso", col_evento]]
            .head()
        )

    return fh


In [ ]:
df = criar_flag_acordo(
    full_history=full_history,
    col_id_hist='id_contrato',
    col_data_ref='data_ref',
    
    rastro_contratos=rastro_contratos,
    col_id_antigo='id_antigo',
    col_evento='data_evento'
    )

📌 criar_flag_acordo — auditoria
• Flags marcadas ...............: 4,123
• Contratos com flag_acordo=1 ..: 4,123


,id_contrato,data_ref,dias_atraso,data_evento
10,10000000,2015-11-01,60,2015-12-01
15,10000001,2015-05-01,90,2015-06-01
23,10000002,2015-08-01,90,2015-09-01
24,10000003,2015-01-01,120,2015-02-01
70,10000007,2015-04-01,90,2015-05-01


In [90]:
# verificação de shape (antes e após mesclagem)
full_history.shape, df.shape

((115186, 5), (115186, 8))

In [91]:
# verificação de fração de contratos com flag_acordo

df['flag_acordo'].value_counts(normalize=True, dropna=False)*100

flag_acordo
0    96.420572
1     3.579428
Name: proportion, dtype: float64

#### Coletando exemplo de contrato que recebeu flag_acordo = 1

In [92]:
df[df['flag_acordo']==1]['id_contrato'].sample(n=3,random_state=0).tolist()

['10003337', '10000061', '10007245']

In [94]:
df[df['id_contrato']=='10003337']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
40398,10003337,10003337,2015-04-29,2015-05-01,201505,30,2016-11-01,0
40399,10003337,10003337,2015-04-29,2015-06-01,201505,30,2016-11-01,0
40400,10003337,10003337,2015-04-29,2015-07-01,201505,60,2016-11-01,0
40401,10003337,10003337,2015-04-29,2015-08-01,201505,60,2016-11-01,0
40402,10003337,10003337,2015-04-29,2015-09-01,201505,60,2016-11-01,0
40403,10003337,10003337,2015-04-29,2015-10-01,201505,60,2016-11-01,0
40404,10003337,10003337,2015-04-29,2015-11-01,201505,90,2016-11-01,0
40405,10003337,10003337,2015-04-29,2015-12-01,201505,60,2016-11-01,0
40406,10003337,10003337,2015-04-29,2016-01-01,201505,60,2016-11-01,0
40407,10003337,10003337,2015-04-29,2016-02-01,201505,60,2016-11-01,0


In [100]:
rastro_contratos[rastro_contratos['id_antigo']=='10003337']

,id_antigo,id_novo,data_evento
2343,10003337,10007348,2016-11-01


In [97]:
full_history[full_history['id_contrato'].isin(['10003337','10007348'])]

,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso
40398,10003337,2015-04-29,2015-05-01,201505,30
40399,10003337,2015-04-29,2015-06-01,201505,30
40400,10003337,2015-04-29,2015-07-01,201505,60
40401,10003337,2015-04-29,2015-08-01,201505,60
40402,10003337,2015-04-29,2015-09-01,201505,60
40403,10003337,2015-04-29,2015-10-01,201505,60
40404,10003337,2015-04-29,2015-11-01,201505,90
40405,10003337,2015-04-29,2015-12-01,201505,60
40406,10003337,2015-04-29,2016-01-01,201505,60
40407,10003337,2015-04-29,2016-02-01,201505,60


In [ ]:
df[df['id_contrato'].isin(['10003337','10007348','10010483'])]

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
40398,10003337,10003337,2015-04-29,2015-05-01,201505,30,2016-11-01,0
40399,10003337,10003337,2015-04-29,2015-06-01,201505,30,2016-11-01,0
40400,10003337,10003337,2015-04-29,2015-07-01,201505,60,2016-11-01,0
40401,10003337,10003337,2015-04-29,2015-08-01,201505,60,2016-11-01,0
40402,10003337,10003337,2015-04-29,2015-09-01,201505,60,2016-11-01,0
40403,10003337,10003337,2015-04-29,2015-10-01,201505,60,2016-11-01,0
40404,10003337,10003337,2015-04-29,2015-11-01,201505,90,2016-11-01,0
40405,10003337,10003337,2015-04-29,2015-12-01,201505,60,2016-11-01,0
40406,10003337,10003337,2015-04-29,2016-01-01,201505,60,2016-11-01,0
40407,10003337,10003337,2015-04-29,2016-02-01,201505,60,2016-11-01,0


### Contratos sem nenhum acordo

In [106]:
# identificar os contratos que tiveram flag_acordo = 1
contratos_com_acordo = df.loc[df["flag_acordo"] == 1, "id_contrato"].unique()

# filtrar todos os que NÃO estão nessa lista
df_sem_acordo = df[~df["id_contrato"].isin(contratos_com_acordo)]

In [107]:
df_sem_acordo['id_contrato'].sample(n=3,random_state=0).tolist()

['10002238', '10003874', '10009286']

In [108]:
df[df['id_contrato']=='10002238']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
26071,10002238,10002238,2014-10-03,2015-01-01,201501,60,NaT,0
26072,10002238,10002238,2014-10-03,2015-02-01,201501,60,NaT,0
26073,10002238,10002238,2014-10-03,2015-03-01,201501,60,NaT,0
26074,10002238,10002238,2014-10-03,2015-04-01,201501,30,NaT,0
26075,10002238,10002238,2014-10-03,2015-05-01,201501,30,NaT,0
26076,10002238,10002238,2014-10-03,2015-06-01,201501,30,NaT,0
26077,10002238,10002238,2014-10-03,2015-07-01,201501,30,NaT,0
26078,10002238,10002238,2014-10-03,2015-08-01,201501,30,NaT,0
26079,10002238,10002238,2014-10-03,2015-09-01,201501,30,NaT,0
26080,10002238,10002238,2014-10-03,2015-10-01,201501,30,NaT,0


In [110]:
df[df['id_contrato']=='10003874']

,id_original,id_contrato,data_inicio_contrato,data_ref,safra,dias_atraso,data_evento,flag_acordo
47123,10002848,10003874,2015-02-06,2015-08-01,201508,0,NaT,0
47124,10002848,10003874,2015-02-06,2015-09-01,201508,0,NaT,0
47125,10002848,10003874,2015-02-06,2015-10-01,201508,0,NaT,0
47126,10002848,10003874,2015-02-06,2015-11-01,201508,0,NaT,0
47127,10002848,10003874,2015-02-06,2015-12-01,201508,0,NaT,0
47128,10002848,10003874,2015-02-06,2016-01-01,201508,0,NaT,0
47129,10002848,10003874,2015-02-06,2016-02-01,201508,0,NaT,0
47130,10002848,10003874,2015-02-06,2016-03-01,201508,0,NaT,0
47131,10002848,10003874,2015-02-06,2016-04-01,201508,15,NaT,0
47132,10002848,10003874,2015-02-06,2016-05-01,201508,30,NaT,0


In [111]:
rastro_contratos[rastro_contratos['id_antigo']=='10003874']

,id_antigo,id_novo,data_evento


### Diagnóstico de Reaging

In [121]:
def diagnosticar_reaging(
    df: pd.DataFrame,
    *,
    col_orig: str = "id_original",
    col_id: str = "id_contrato",
    col_date: str = "data_ref",
    col_delay: str = "dias_atraso",
    col_flag: str = "flag_acordo",
    verbose: bool = True,
) -> dict:
    """
    Diagnostica re-aging em nível de cadeia (`id_original`).

    • Um *evento* de acordo é cada linha com flag_acordo = 1.  
    • Um evento é considerado re-aging quando:
        1. atraso_antes > 0  (até o mês da flag, inclusive) e
        2. atraso_depois = 0 (após o mês da flag).

    Classificação de cada cadeia
    ----------------------------
    - COM re-aging ........: possui ≥1 evento que atende aos critérios.
    - SEM re-aging ........: possui eventos, mas nenhum atende aos critérios.
    - COM *e* SEM re-aging : possui ≥1 evento re-aging **e** ≥1 evento não-re-aging.
    """
    # ── 1. Pré-processamento ───────────────────────────────────────────
    df = df[[col_orig, col_id, col_date, col_delay, col_flag]].copy()
    df[col_date] = pd.to_datetime(df[col_date]).dt.normalize()
    df.sort_values([col_orig, col_date], inplace=True)

    cadeias = df.loc[df[col_flag] == 1, col_orig].unique()
    stats = {
        "com_re": [],        # só re-aging
        "sem_re": [],        # só não-re-aging
        "mixed": []          # ambos
    }

    # ── 2. Avalia cada cadeia de contratos ────────────────────────────
    for cid in cadeias:
        g = df[df[col_orig] == cid]
        eventos = g[g[col_flag] == 1][col_date]

        reaging_flags = []
        for evento_dt in eventos:
            atraso_antes  = g[g[col_date] <= evento_dt][col_delay].max()
            atraso_depois = g[g[col_date] >  evento_dt][col_delay].min() \
                            if not g[g[col_date] >  evento_dt].empty else None
            reaging_flags.append((atraso_antes > 0) and (atraso_depois == 0))

        if all(reaging_flags):
            stats["com_re"].append(cid)
        elif not any(reaging_flags):
            stats["sem_re"].append(cid)
        else:
            stats["mixed"].append(cid)

    totais = {k: len(v) for k, v in stats.items()}
    total_cadeias = sum(totais.values())

    # ── 3. Seleciona cadeias-exemplo para exibição ─────────────────────
    def pick(df_base, ids):
        if not ids:
            return pd.DataFrame(), None
        # pega a primeira apenas para inspeção
        cid = ids[0]
        return df_base[df_base[col_orig] == cid], cid

    ex_com_re , id_com_re  = pick(df, stats["com_re"])
    ex_sem_re , id_sem_re  = pick(df, stats["sem_re"])
    ex_mixed  , id_mixed   = pick(df, stats["mixed"])

    # ── 4. Auditoria ──────────────────────────────────────────────────
    if verbose:
        print("📊 Re-aging por cadeia (id_original)")
        print(f"• Total de cadeias avaliadas ...........: {total_cadeias:,}")
        print(f"  ├─ COM re-aging ......................: {totais['com_re']:,}")
        print(f"  ├─ SEM re-aging ......................: {totais['sem_re']:,}")
        print(f"  └─ COM e SEM (mistas) ...............: {totais['mixed']:,}\n")

        if not ex_com_re.empty:
            print(f"🔍 Cadeia COM re-aging (id_original = {id_com_re})")
            display(ex_com_re[[col_orig, col_id, col_date,
                               col_delay, col_flag]])

        if not ex_sem_re.empty:
            print(f"\n🔍 Cadeia SEM re-aging (id_original = {id_sem_re})")
            display(ex_sem_re[[col_orig, col_id, col_date,
                               col_delay, col_flag]])

        if not ex_mixed.empty:
            print(f"\n🔍 Cadeia COM e SEM re-aging (id_original = {id_mixed})")
            display(ex_mixed[[col_orig, col_id, col_date,
                              col_delay, col_flag]])

    # ── 5. Retorno estruturado ────────────────────────────────────────
    return {
        "totais": totais,
        "exemplo_com":  ex_com_re,
        "exemplo_sem":  ex_sem_re,
        "exemplo_misto": ex_mixed
    }


In [125]:
resultado = diagnosticar_reaging(df)

📊 Re-aging por cadeia (id_original)
• Total de cadeias avaliadas ...........: 3,364
  ├─ COM re-aging ......................: 3,326
  ├─ SEM re-aging ......................: 30
  └─ COM e SEM (mistas) ...............: 8

🔍 Cadeia COM re-aging (id_original = 10000000)


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo
0,10000000,10000000,2015-01-01,30,0
1,10000000,10000000,2015-02-01,15,0
2,10000000,10000000,2015-03-01,15,0
3,10000000,10000000,2015-04-01,30,0
4,10000000,10000000,2015-05-01,60,0
5,10000000,10000000,2015-06-01,90,0
6,10000000,10000000,2015-07-01,60,0
7,10000000,10000000,2015-08-01,90,0
8,10000000,10000000,2015-09-01,90,0
9,10000000,10000000,2015-10-01,60,0



🔍 Cadeia SEM re-aging (id_original = 10000446)


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo
4279,10000446,10000446,2015-01-01,0,0
4280,10000446,10000446,2015-02-01,0,0
4281,10000446,10000446,2015-03-01,0,1
35209,10000446,10002926,2015-04-01,0,0



🔍 Cadeia COM e SEM re-aging (id_original = 10000271)


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo
2650,10000271,10000271,2015-01-01,0,0
2651,10000271,10000271,2015-02-01,0,0
2652,10000271,10000271,2015-03-01,0,0
2653,10000271,10000271,2015-04-01,0,1
37814,10000271,10003137,2015-05-01,0,0
37815,10000271,10003137,2015-06-01,0,0
37816,10000271,10003137,2015-07-01,0,0
37817,10000271,10003137,2015-08-01,0,0
37818,10000271,10003137,2015-09-01,0,0
37819,10000271,10003137,2015-10-01,15,0


# Construção de Over90

In [ ]:
# Cria coluna over90
df['over90'] = np.where(df['dias_atraso'] >= 90, 1, 0)

In [ ]:
# Distribuição formatada
print("📊 Distribuição da coluna 'over90':")
dist = df['over90'].value_counts(normalize=True, dropna=False).mul(100).round(2)
display(dist.rename("Percentual (%)").to_frame())

# Exemplo de registros com over90 = 1
print("\n🔍 Exemplos de contratos com over90 = 1 (dias_atraso ≥ 90):")
cols_exibir = ["id_original", "id_contrato", "data_ref", "dias_atraso", "flag_acordo", "over90"]
display(df[df["over90"] == 1][cols_exibir].head())


📊 Distribuição da coluna 'over90':


,Percentual (%)
over90,
0,83.33
1,16.67



🔍 Exemplos de contratos com over90 = 1 (dias_atraso ≥ 90):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90
5,10000000,10000000,2015-06-01,90,0,1
7,10000000,10000000,2015-08-01,90,0,1
8,10000000,10000000,2015-09-01,90,0,1
15,10000001,10000001,2015-05-01,90,1,1
23,10000002,10000002,2015-08-01,90,1,1


# Construção de Mau

In [131]:
df["mau"] = np.where((df["flag_acordo"] == 1) | (df["over90"] == 1), 1, 0)

In [132]:
# Exibe distribuição geral
print("📊 Distribuição da coluna 'mau':")
print((df["mau"].value_counts(normalize=True, dropna=False) * 100)
      .rename({1: "mau = 1", 0: "mau = 0"}).round(2))
print()

📊 Distribuição da coluna 'mau':
mau
mau = 0    82.09
mau = 1    17.91
Name: proportion, dtype: float64



In [134]:
# Amostras específicas

# 1. Garante exclusividade dos grupos
ex_ambos   = df[(df["mau"] == 1) & (df["flag_acordo"] == 1) & (df["over90"] == 1)].copy()
ex_acordo  = df[(df["mau"] == 1) & (df["flag_acordo"] == 1) & (df["over90"] == 0)].copy()
ex_over90  = df[(df["mau"] == 1) & (df["flag_acordo"] == 0) & (df["over90"] == 1)].copy()

# 2. Contagens absolutas
n_total = df["mau"].sum()
n_acordo = len(ex_acordo)
n_over90 = len(ex_over90)
n_ambos  = len(ex_ambos)

# 3. Percentuais
pct_acordo = 100 * n_acordo / n_total
pct_over90 = 100 * n_over90 / n_total
pct_ambos  = 100 * n_ambos  / n_total

# 4. Exibição de amostras
cols_exibir = ["id_original", "id_contrato", "data_ref", "dias_atraso", "flag_acordo", "over90", "mau"]

print("🔹 Exemplos de 'mau' causados apenas por acordo (flag_acordo = 1 e over90 = 0):")
display(ex_acordo.head(5)[cols_exibir])

print("🔹 Exemplos de 'mau' causados apenas por atraso >= 90 dias (over90 = 1 e flag_acordo = 0):")
display(ex_over90.head(5)[cols_exibir])

print("🔹 Exemplos de 'mau' causados por acordo e atraso >= 90 dias (ambos = 1):")
display(ex_ambos.head(5)[cols_exibir])

# 5. Sumário
print("\n📊 Distribuição dos motivos para 'mau' (total: {:,})".format(n_total))
print(f"• Apenas acordo ...............: {n_acordo:,} registros ({pct_acordo:.2f}%)")
print(f"• Apenas atraso >= 90 dias ....: {n_over90:,} registros ({pct_over90:.2f}%)")
print(f"• Ambos (acordo e atraso ≥ 90): {n_ambos:,} registros ({pct_ambos:.2f}%)")


🔹 Exemplos de 'mau' causados apenas por acordo (flag_acordo = 1 e over90 = 0):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90,mau
10,10000000,10000000,2015-11-01,60,1,0,1
90,10000009,10000009,2016-02-01,60,1,0,1
99,10000010,10000010,2015-09-01,60,1,0,1
130,10000015,10000015,2015-04-01,60,1,0,1
163,10000017,10000017,2016-10-01,60,1,0,1


🔹 Exemplos de 'mau' causados apenas por atraso >= 90 dias (over90 = 1 e flag_acordo = 0):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90,mau
5,10000000,10000000,2015-06-01,90,0,1,1
7,10000000,10000000,2015-08-01,90,0,1,1
8,10000000,10000000,2015-09-01,90,0,1,1
67,10000007,10000007,2015-01-01,180,0,1,1
68,10000007,10000007,2015-02-01,120,0,1,1


🔹 Exemplos de 'mau' causados por acordo e atraso >= 90 dias (ambos = 1):


,id_original,id_contrato,data_ref,dias_atraso,flag_acordo,over90,mau
15,10000001,10000001,2015-05-01,90,1,1,1
23,10000002,10000002,2015-08-01,90,1,1,1
24,10000003,10000003,2015-01-01,120,1,1,1
70,10000007,10000007,2015-04-01,90,1,1,1
101,10000011,10000011,2015-02-01,90,1,1,1



📊 Distribuição dos motivos para 'mau' (total: 20,633)
• Apenas acordo ...............: 1,431 registros (6.94%)
• Apenas atraso >= 90 dias ....: 16,510 registros (80.02%)
• Ambos (acordo e atraso ≥ 90): 2,692 registros (13.05%)


In [3]:
# PASSO 1: Gerar o conjunto de dados robusto
# (10.000 contratos * 24 meses = 240.000 registros)
dados = pd.read_excel('gabarito_ever_over.xlsx')

# Mostra informações sobre o DataFrame gerado
print("\n--- Informações do DataFrame Gerado ---")
dados.info()
print("\n--- Primeiros Registros ---")
display(dados.head())
print("\n--- Últimos Registros ---")
display(dados.tail())


--- Informações do DataFrame Gerado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   safra_ref                  36 non-null     int64
 1   id_contrato                36 non-null     int64
 2   atraso                     36 non-null     int64
 3   gabarito_over90            36 non-null     int64
 4   gabarito_over90m12         36 non-null     int64
 5   gabarito_ever90m12         36 non-null     int64
 6   flag_acordo                36 non-null     int64
 7   gabarito_bad90             36 non-null     int64
 8   gabarito_target_ever90m12  36 non-null     int64
 9   gabarito_target_over90m12  36 non-null     int64
dtypes: int64(10)
memory usage: 2.9 KB

--- Primeiros Registros ---


,safra_ref,id_contrato,atraso,gabarito_over90,gabarito_over90m12,gabarito_ever90m12,flag_acordo,gabarito_bad90,gabarito_target_ever90m12,gabarito_target_over90m12
0,202001,1000010,0,0,0,0,0,0,0,0
1,202002,1000010,0,0,0,0,0,0,1,1
2,202003,1000010,0,0,0,0,0,0,1,0
3,202004,1000010,15,0,1,1,0,0,1,1
4,202005,1000010,45,0,0,1,0,0,1,0



--- Últimos Registros ---


,safra_ref,id_contrato,atraso,gabarito_over90,gabarito_over90m12,gabarito_ever90m12,flag_acordo,gabarito_bad90,gabarito_target_ever90m12,gabarito_target_over90m12
31,202208,1000010,5,0,0,1,0,0,1,0
32,202209,1000010,15,0,0,1,0,0,1,0
33,202210,1000010,30,0,0,1,0,0,1,0
34,202211,1000010,60,0,0,1,0,0,1,0
35,202212,1000010,90,1,0,0,0,1,0,0


In [3]:
# 2. Defina as metas que você quer calcular
#    Formato: (tipo, dias_de_atraso, horizonte_em_meses)
minhas_metas = [
    ('ever', 90, 12),   # Atingiu 90 dias de atraso em algum momento nos próximos 12 meses?
    ('over', 90, 12),    # Estava com 90 dias ou mais de atraso exatamente 12 meses depois?
    ('ever', 30, 4),    # Atingiu 30 dias de atraso em algum momento nos próximos 4 meses?
    ('over', 30, 4),    # Estava com 30 dias ou mais de atraso exatamente 4 meses depois?
]

# 3. Crie a calculadora e execute o cálculo
#    Instancie a classe com suas especificações.
calculadora = CalculadoraTargets(
    specs=minhas_metas, 
    date_col='safra_ref',
    contract_col='id_contrato',
    dpd_col='atraso'
    )

In [4]:
minhas_metas = [
    ('ever', 90, 12),   
    ('over', 90, 12),   
    ('ever', 30, 4),    
    ('over', 30, 4),    
]

In [5]:
#    Chame o método 'calcular' para obter o resultado.
df = calculadora.calcular(dados)

# 4. Veja o resultado!
display(df[['safra_ref','id_contrato','atraso','over90m12', 'gabarito_over90m12']])
#display(df[['safra_ref','id_contrato','atraso','ever90m12', 'gabarito_ever90m12']])

,safra_ref,id_contrato,atraso,over90m12,gabarito_over90m12
0,202001,1000010,0,0,0
1,202002,1000010,0,0,0
2,202003,1000010,0,0,0
3,202004,1000010,15,1,1
4,202005,1000010,45,0,0
5,202006,1000010,75,0,0
6,202007,1000010,0,0,0
7,202008,1000010,0,0,0
8,202009,1000010,0,0,0
9,202010,1000010,0,0,0


In [6]:
minhas_metas = [
    ('ever', 90, 12),   
    ('over', 90, 12),   
    #('ever', 30, 4),    
    #('over', 30, 4),    
]

calculadora = CalculadoraTargets(
    specs=minhas_metas, 
    date_col='safra_ref',
    contract_col='id_contrato',
    dpd_col='atraso'
    )

df_targets = calculadora.calcular(dados)

# exclui meses não maturados (comportamento default)
fig1 = calculadora.plot_targets(df_targets,
                         ["target_ever90m12", "target_over30m4"])
fig1.show()

# plota séries completas, mesmo que incompletas
fig2 = calculadora.plot_targets(df_targets,
                         ["target_ever90m12", "target_over30m4"],
                         drop_unmatured=False)
fig2.show()


KeyError: 'Column not found: target_ever90m12'

In [ ]:
minhas_metas = [
    ('ever', 90, 12),   
    ('over', 90, 12),   
    #('ever', 30, 4),    
    #('over', 30, 4),    
]

calculadora = CalculadoraTargets(
    specs=minhas_metas, 
    date_col='safra_ref',
    contract_col='id_contrato',
    dpd_col='atraso'
    )

df_work = dados.copy()
# 1) garante coluna _date interna
df_work["_date"] = pd.to_datetime(df_work["safra_ref"].astype(str), format="%Y%m")

# 2) flags “bad”
calculadora._flag_bad_columns(df_work, flag_col="flag_acordo")

# 3) targets
calculadora._calculate_ever_targets_from_bad(df_work)
calculadora._calculate_over_targets_from_bad(df_work)

# pronto!  df_work agora tem:
#   bad30, bad90, target_ever90m12, target_over90m12, target_ever30m4, target_over30m4


In [7]:
display(df_work[['safra_ref','id_contrato','atraso','flag_acordo', 'target_ever90m12', 'gabarito_target_ever90m12']])

,safra_ref,id_contrato,atraso,flag_acordo,target_ever90m12,gabarito_target_ever90m12
0,202001,1000010,0,0,0,0
1,202002,1000010,0,0,1,1
2,202003,1000010,0,0,1,1
3,202004,1000010,15,0,1,1
4,202005,1000010,45,0,1,1
5,202006,1000010,75,0,1,1
6,202007,1000010,0,0,1,1
7,202008,1000010,0,0,1,1
8,202009,1000010,0,0,1,1
9,202010,1000010,0,0,1,1


In [8]:
display(df_work[['safra_ref','id_contrato','atraso','flag_acordo', 'target_over90m12', 'gabarito_target_over90m12']])

,safra_ref,id_contrato,atraso,flag_acordo,target_over90m12,gabarito_target_over90m12
0,202001,1000010,0,0,0,0
1,202002,1000010,0,0,1,1
2,202003,1000010,0,0,0,0
3,202004,1000010,15,0,1,1
4,202005,1000010,45,0,0,0
5,202006,1000010,75,0,0,0
6,202007,1000010,0,0,0,0
7,202008,1000010,0,0,0,0
8,202009,1000010,0,0,0,0
9,202010,1000010,0,0,0,0


In [ ]:
fig = calculadora.plot_targets(
    df,
    target_cols=[
        "ever90m12",
        "over90m12",
        "ever30m4",
        "over30m4",
        ],
    normalize=True,   # exibe porcentagem
    title="Evolução dos targets"
)
fig.show()


In [ ]:
df[df['ever90m12']==1]

In [ ]:
# cálculo dos targets

# ------------------- auditoria de contrato escolhido manualmente ---------------
evid1 = calculadora.audit_contract(
    df,
    contract_id="00000010",           # substitua pelo seu CCB real
    show_only_mismatches=False        # veja todas as linhas, com coluna __match__
)
display(evid1.head())

In [ ]:
# ------------------- auditoria de contrato aleatório ---------------------------
evid2 = calculadora.audit_contract(
    df,
    random_state=42                   # para reproduzir a mesma escolha
    # contract_id fica None → aleatório
)
if evid2.empty:
    print("Nenhuma divergência encontrada!")
else:
    print("Divergências:", evid2)
